# Rozpoznawanie jedzenia AI

### Przygotowanie dysku google

In [ ]:
#!pip install scikit-learn==1.0.2
!pip install scikeras
!pip install --upgrade scikeras scikit-learn
# !pip uninstall scikeras -y
# !pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 106.5 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2


In [ ]:
import os

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Model 3

### Instalacja Kaggle API


In [ ]:
!pip install kaggle

!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/Programowanie/Food/KaggleToken/kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

### Instalacja tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scikeras.wrappers import KerasClassifier


### Pobranie i rozpakowanie zbioru danych

In [ ]:
dataset_path = '/content/food-101'

if not os.path.exists(dataset_path):
    print("Downloading dataset...")
    !kaggle datasets download -d kmader/food41 -p /content/ > /dev/null
    print("Extracting dataset...")
    !unzip /content/food41.zip -d /content/food-101 > /dev/null
    print("Dataset downloaded and extracted.")
else:
    print("Dataset already exists.")

Dataset already exists.


### Przygotowanie danych

In [ ]:
def create_dataset(data_dir, subset):
    dataset = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        labels='inferred',
        label_mode='categorical',
        image_size=(224, 224),
        interpolation='nearest',
        batch_size=64,
        shuffle=True,
        validation_split=0.2,
        subset=subset,
        seed=123
    )
    return dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

train_data = create_dataset(dataset_path + '/images', subset='training')
val_data = create_dataset(dataset_path + '/images', subset='validation')

Found 101000 files belonging to 101 classes.
Using 80800 files for training.
Found 101000 files belonging to 101 classes.
Using 20200 files for validation.


### Import niezbędnych bibliotek


In [ ]:
#model = tf.keras.models.load_model(checkpoint_path)


### Ścieżka do zapisu modelu


## Trening modelu

### Model 1

In [ ]:
checkpoint_path = '/content/drive/MyDrive/Programowanie/Food/Models/efficientnet_model_checkpoint.keras'



In [ ]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(101, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=3,  # Wstrzymanie treningu po 3 epokach bez poprawy
    restore_best_weights=True,
    verbose=1
)

checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=False,
    mode='max',
    verbose=2
)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling_2 (Rescaling)   │ (None, 224, 224, 3)    │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ normalization_1           │ (None, 224, 224, 3)    │              7 │ rescaling_2[0][0]      │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling_3 (Rescaling)   │ (None, 224, 224, 3)    │              0 │ normalization_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv_pad             │ (None, 225, 225, 3)    │              0 │ rescaling_3[0][0]      │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv (Conv2D)        │ (None, 112, 112, 32)   │            864 │ stem_conv_pad[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_bn                   │ (None, 112, 112, 32)   │            128 │ stem_conv[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_activation           │ (None, 112, 112, 32)   │              0 │ stem_bn[0][0]          │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_dwconv            │ (None, 112, 112, 32)   │            288 │ stem_activation[0][0]  │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_bn                │ (None, 112, 112, 32)   │            128 │ block1a_dwconv[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_activation        │ (None, 112, 112, 32)   │              0 │ block1a_bn[0][0]       │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_squeeze        │ (None, 32)             │              0 │ block1a_activation[0]… │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reshape        │ (None, 1, 1, 32)       │              0 │ block1a_se_squeeze[0]… │
│ (Reshape)                 │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reduce         │ (None, 1, 1, 8)        │            264 │ block1a_se_reshape[0]… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 4,403,464 (16.80 MB)

 Trainable params: 353,893 (1.35 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    callbacks=[early_stopping, checkpoint_callback]
)

base_model.trainable = True 

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_fine_tune = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5, 
    callbacks=[early_stopping, checkpoint_callback]
)

Epoch 1/10
1263/1263 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.3358 - loss: 2.7947
Epoch 1: val_accuracy improved from -inf to 0.59579, saving model to /content/drive/MyDrive/Programowanie/Food/Models/efficientnet_model_checkpoint.keras
1263/1263 ━━━━━━━━━━━━━━━━━━━━ 175s 117ms/step - accuracy: 0.3358 - loss: 2.7943 - val_accuracy: 0.5958 - val_loss: 1.5737
Epoch 2/10
1262/1263 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.5256 - loss: 1.8274
Epoch 2: val_accuracy improved from 0.59579 to 0.61906, saving model to /content/drive/MyDrive/Programowanie/Food/Models/efficientnet_model_checkpoint.keras
1263/1263 ━━━━━━━━━━━━━━━━━━━━ 96s 76ms/step - accuracy: 0.5256 - loss: 1.8274 - val_accuracy: 0.6191 - val_loss: 1.4696
Epoch 3/10
1262/1263 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.5567 - loss: 1.7040
Epoch 3: val_accuracy improved from 0.61906 to 0.62777, saving model to /content/drive/MyDrive/Programowanie/Food/Models/efficientnet_model_checkpoint.keras
1263/1263 ━━━━━━━━━━━

powyższy model ma:
- accuracy: 0.6267
- loss: 1.3999
- val_accuracy: 0.6973
- val_loss: 1.1628

### Model 2

In [ ]:
checkpoint_path = '/content/drive/MyDrive/Programowanie/Food/Models/model_2_checkpoint.keras'



In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
import numpy as np


def create_model(dropout_rate=0.3, learning_rate=0.001, dense_units=256):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    for layer in base_model.layers[-20:]:
        layer.trainable = True

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(dense_units, activation='relu')(x) 
    x = Dropout(dropout_rate)(x)
    predictions = Dense(101, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_data = datagen.flow_from_directory(
    dataset_path + '/images',  
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='training',
    seed=123
)


model = KerasClassifier(build_fn=create_model, verbose=0)
param_grid = {
    'dropout_rate': [0.2, 0.3, 0.4, 0.5],
    'learning_rate': [1e-3, 1e-4, 1e-5],
    'dense_units': [128, 256, 512], 
    'epochs': [10, 20, 30] 
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3)



Found 101000 images belonging to 101 classes.


In [ ]:
X_train, y_train = [], []
for images, labels in train_data:
    for i in range(images.shape[0]):
        X_train.append(images[i]) 
        y_train.append(labels[i]) 

X_train = np.array(X_train)
y_train = np.array(y_train)
X_train = np.array(X_train)
y_train = np.array(y_train)

grid_result = grid.fit(X_train, y_train)

print("Best parameters: %s" % grid_result.best_params_)
print("Best score: %f" % grid_result.best_score_)

best_model = create_model(**grid_result.best_params_)

early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True, verbose=1)
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=False,
    mode='max',
    verbose=2
)

history = best_model.fit(
    train_data,
    validation_data=val_data,
    epochs=grid_result.best_params_['epochs'],
    callbacks=[early_stopping, checkpoint_callback]
)